Code to preprocess the IHDP data and generate the dataset used in our experiments.

In [ ]:
import os
import pandas as pd
import numpy as np

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects.conversion import localconverter

In [ ]:
DATA_DIR = ""

In [ ]:
def remove_dict_key(dict_input, key):
  dict_copy = copy.deepcopy(dict_input)
  del dict_copy[key]
  return dict_copy

def remove_dict_keys(dict_input, keys):
  for key in keys:
    dict_input = remove_dict_key(dict_input, key)
  return dict_input

def get_sum_of_squares_matrix(df):
  return df.transpose().dot(df)

In [ ]:
pandas2ri.activate()

base = importr('base')

In [ ]:
base.load(os.path.join(DATA_DIR, "ihdp.RData"))

'ihdp'


In [ ]:
df_raw = base.mget(base.ls())[0]
df_raw.head()

,treat,bw,b.head,preterm,birth.o,nnhealth,momage,sex,twin,b.marr,mom.lths,mom.hs,mom.scoll,cig,first,booze,drugs,work.dur,prenatal,ark,ein,har,mia,pen,tex,was,momwhite,momblack,momhisp
1,1,1559,28.648521,10,2,94,33,1,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0,1,0,0
2,1,1420,27.000000,4,2,85,15,1,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0,0,0,0,1,0
3,0,1000,25.000000,8,4,89,33,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,1,0
4,0,1430,29.000000,6,1,112,22,0,0,0,1,0,0,0,2,0,1,0,1,1,0,0,0,0,0,0,0,1,0
5,0,1984,31.000000,2,1,99,20,0,0,0,0,0,1,1,2,0,1,0,1,1,0,0,0,0,0,0,0,1,0


In [ ]:
def create_observational_data_from(df_raw):
  # Drop a non-random subset of the data to convert the experimental dataset to
  # an observational one.
  df = df_raw[(
      np.logical_or(
          df_raw['treat'] == 0,
          np.logical_and(df_raw['treat'] == 1, df_raw['momwhite'] == 1)))]
  
  return df.drop(columns=['momwhite', 'momblack', 'momhisp'])

df_obs = create_observational_data_from(df_raw)
df_obs.head()

,treat,bw,b.head,preterm,birth.o,nnhealth,momage,sex,twin,b.marr,mom.lths,mom.hs,mom.scoll,cig,first,booze,drugs,work.dur,prenatal,ark,ein,har,mia,pen,tex,was
1,1,1559,28.648521,10,2,94,33,1,0,1,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,0
3,0,1000,25.000000,8,4,89,33,0,0,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0
4,0,1430,29.000000,6,1,112,22,0,0,0,1,0,0,0,2,0,1,0,1,1,0,0,0,0,0,0
5,0,1984,31.000000,2,1,99,20,0,0,0,0,0,1,1,2,0,1,0,1,1,0,0,0,0,0,0
6,0,1320,28.000000,7,2,110,23,1,0,0,0,0,1,1,1,0,1,1,1,1,0,0,0,0,0,0


In [ ]:
def standardize_columns(df, columns):
  df = df.copy(deep=True)

  for col in columns:
    df["%s_std" % col] = (df[col] - np.mean(df[col])) / np.std(df[col])
  
  return df

In [ ]:
df_final = standardize_columns(df_obs[["treat", "bw", "cig"]], ["bw"])
df_final.head()

,treat,bw,cig,bw_std
1,1,1559,0,-0.528957
3,0,1000,1,-1.738109
4,0,1430,0,-0.807992
5,0,1984,1,0.390344
6,0,1320,1,-1.045929


In [ ]:
df_final.to_csv(os.path.join(DATA_DIR, "ihdp_processed.csv"), index=False)